In [7]:
IP Address: 192.168.56.2

INFO:root:SalesInvoice table created.
ERROR:root:Error inserting data into PostgreSQL: there is no unique or exclusion constraint matching the ON CONFLICT specification

INFO:root:PostgreSQL connection closed.


In [ ]:
import psycopg2
import pandas as pd
import requests
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)  # Adjust level as needed

# Function to check if table exists
def table_exists(cursor, table_name):
    cursor.execute("""
    SELECT EXISTS (
       SELECT 1
       FROM   information_schema.tables 
       WHERE  table_schema = 'public'
       AND    table_name = %s
    );
    """, (table_name,))
    return cursor.fetchone()[0]

# Define connection details for PostgreSQL
db_config = {
    'host': 'localhost',
    'user': 'postgres',
    'password': 'admin@123',
    'dbname': 'postgres'
}

# Function to handle data retrieval and insertion
def fetch_and_insert_data(url, params, headers, insert_query, cursor):
    limit_start = 0
    limit_page_length = 1000
    all_data = []
    while True:
        params['limit_start'] = limit_start
        try:
            response = requests.get(url, params=params, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            if 'data' in data:
                current_page_data = data['data']
                all_data.extend(current_page_data)
                if len(current_page_data) < limit_page_length:
                    break 
                else:
                    limit_start += limit_page_length  
            else:
                break  
        except requests.exceptions.RequestException as e:
            logging.error(f"Error fetching data: {e}")
            break

    # Normalize JSON data into a DataFrame
    invoice = pd.json_normalize(all_data)
    invoice.rename(columns={'base_grand_total': 'grand_total'}, inplace=True)
    invoice = invoice[invoice["credit_rating_not_to_consider"] != 1]

    # Insert data into PostgreSQL
    try:
        for row in invoice.itertuples(index=False):
            cursor.execute(insert_query, (
                row.name,
                row.customer,
                row.grand_total,
                row.due_date,
                row.posting_date,
                row.outstanding_amount,
                bool(row.credit_rating_not_to_consider)  
            ))
        connection.commit()
        logging.info("Data insertion successful.")
    except psycopg2.Error as e:
        logging.error(f"Error inserting data into PostgreSQL: {e}")
        connection.rollback()  # Rollback changes in case of error

# Connect to PostgreSQL
try:
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()

    # Define table name
    table_name = 'SalesInvoice'

    # Check if table exists, create if it doesn't
    if not table_exists(cursor, table_name):
        create_table_query = """
        CREATE TABLE IF NOT EXISTS SalesInvoice (
            name VARCHAR(255) PRIMARY KEY,
            customer VARCHAR(255),
            grand_total DECIMAL(15, 2),
            due_date DATE,
            posting_date DATE,
            outstanding_amount DECIMAL(15, 2),
            credit_rating_not_to_consider BOOLEAN
        )
        """
        cursor.execute(create_table_query)
        logging.info("SalesInvoice table created.")

    # Define API endpoint and parameters
    base_url = 'https://erpv14.electrolabgroup.com/'
    endpoint = 'api/resource/Sales Invoice'
    url = base_url + endpoint
    params = {
        'fields': '["name","customer","base_grand_total","due_date","posting_date","outstanding_amount","credit_rating_not_to_consider"]',
        'limit_page_length': 1000,
        'filters': '[["company", "=", "Electrolab India Pvt. Ltd."]]'
    }
    headers = {
        'Authorization': 'token 3ee8d03949516d0:6baa361266cf807'
    }

    # Define INSERT query with conflict resolution
    insert_query = """
    INSERT INTO SalesInvoice (name, customer, grand_total, due_date, posting_date, outstanding_amount, credit_rating_not_to_consider)
    VALUES (%s, %s, %s, %s, %s, %s, %s::BOOLEAN)
    ON CONFLICT (name) DO UPDATE
    SET
        customer = EXCLUDED.customer,
        grand_total = EXCLUDED.grand_total,
        due_date = EXCLUDED.due_date,
        posting_date = EXCLUDED.posting_date,
        outstanding_amount = EXCLUDED.outstanding_amount,
        credit_rating_not_to_consider = EXCLUDED.credit_rating_not_to_consider
    """

    # Fetch data and insert into PostgreSQL
    fetch_and_insert_data(url, params, headers, insert_query, cursor)

except psycopg2.Error as e:
    logging.error(f"Error connecting to PostgreSQL: {e}")

finally:
    # Close cursor and connection
    if 'connection' in locals():
        cursor.close()
        connection.close()
        logging.info("PostgreSQL connection closed.")


In [8]:
import psycopg2

# Define connection details for PostgreSQL
db_config = {
    'host': 'localhost',  # Use the IP address or hostname of your PostgreSQL server
    'user': 'postgres',
    'password': 'admin@123',
    'dbname': 'postgres'
}

# Connect to PostgreSQL
connection = psycopg2.connect(**db_config)

# Create a cursor object
cursor = connection.cursor()

# Retrieve and display all tables in the database
cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
""")

tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])

# Close the cursor and connection
cursor.close()
connection.close()


Tables in the database:
salesinvoice
